## Plotting practice and some NLP (hopefully)

IMPORTANT NOTE:

subtitles in each season differ in terms of who the provider is. So, basically, there's no guarantee that SDH comments are consistent (each episode in each season has sounds / different sounds can be named differently)

In [193]:
import re
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta, time

import plotly.express as px

In [234]:
data = pd.read_csv('sdh_comments.csv')

# changing string type to datetime
data.interval_start = pd.to_datetime(data.interval_start.str[0:8],format="%H:%M:%S")
data.interval_end = pd.to_datetime(data.interval_end.str[0:8],format="%H:%M:%S")

# this doesn't work with the plotly gantt chart down below
# data.interval_start = pd.to_datetime(data.interval_start.str[0:8],format="%H:%M:%S").dt.time
# data.interval_end = pd.to_datetime(data.interval_end.str[0:8],format="%H:%M:%S").dt.time

In [235]:
data.dtypes

sdh_comment                  object
interval_start       datetime64[ns]
interval_end         datetime64[ns]
interval_minute               int64
episode                       int64
season                        int64
season00episode00            object
dtype: object

Practicing on Season 1 data:

In [196]:
# choosing season 1 and comments containing "musik"
# later we'll make clusters for comments to display different types of sounds: clicks, peeps, thunder, music, radio and so on
data_s01e01 = data[(data.season==1) & (data.sdh_comment.str.lower().str.contains('musik'))]
# |klavierspiel|klänge
data_s01e01

,sdh_comment,interval_start,interval_end,interval_minute,episode,season,season00episode00
103,ruhige Klaviermusik,1900-01-01 00:00:06,1900-01-01 00:00:08,0,7,1,S01E07
105,ruhige spieluhrartige Musik,1900-01-01 00:00:17,1900-01-01 00:00:20,0,7,1,S01E07
106,Musik wird zunehmend dissonant,1900-01-01 00:00:27,1900-01-01 00:00:29,0,7,1,S01E07
110,ruhige beklemmende Musik,1900-01-01 00:02:39,1900-01-01 00:02:42,2,7,1,S01E07
111,Ruhige beklemmende Musik läuft weiter,1900-01-01 00:02:51,1900-01-01 00:02:53,2,7,1,S01E07
...,...,...,...,...,...,...,...
2433,düstere Musik,1900-01-01 00:29:39,1900-01-01 00:29:41,29,9,1,S01E09
2435,Musik verstummt,1900-01-01 00:29:59,1900-01-01 00:30:01,29,9,1,S01E09
2467,melancholische Musik,1900-01-01 00:40:46,1900-01-01 00:40:48,40,9,1,S01E09
2473,melancholische Musik,1900-01-01 00:45:51,1900-01-01 00:45:53,45,9,1,S01E09


In [197]:
fig = px.timeline(data_s01e01, x_start='interval_start',
                            x_end='interval_end',
                            y='season00episode00',
                            # color='sdh_comment',
                            text='sdh_comment'
                            # ,template="xgridoff"
                            ,template='plotly_dark'
                            ,color_discrete_sequence=["rgb(120,130,90)"]                            
                            ,hover_name="sdh_comment"
                            ,hover_data={'sdh_comment':False,'interval_end':False}
                            ,height=600
                            )

fig.update_xaxes(tickformat="%H:%M:%S",
                showgrid=True, griddash='dot', gridcolor='grey',
                rangeslider=dict(visible=True,
                # ,bgcolor="white"
                )
                ,rangeslider_thickness = 0.1 #fraction of the whole fig heigth
                )
# https://plotly.com/python/reference/layout/xaxis/#layout-xaxis-rangeslider

fig.update_yaxes(autorange="reversed",categoryorder='category ascending')

# shapes were added in the background for more readability
for episode in data_s01e01.season00episode00.drop_duplicates():
    fig.add_hrect(y0=episode, y1=episode, line_width=30, fillcolor="black", opacity=0.03,layer="below")

fig.update_traces(textfont_size=8, textangle=0)
fig.update_layout(title="Music of Dark",xaxis_title="Time",yaxis_title="Episode",bargap=0,
                font_family="Courier New",
                font_color="white",
                # title_font_family="Times New Roman",
                # title_font_color="white"
                )
fig.show()

In [198]:
# idea: make color=season, plot based on episode number only
data = data[data.sdh_comment.str.lower().str.contains('musik')]
data.episode = data.episode.astype('str')
data.season = data.season.astype('str')
# |klavierspiel|klänge
data

,sdh_comment,interval_start,interval_end,interval_minute,episode,season,season00episode00
0,düster-melancholische Musik,1900-01-01 00:00:13,1900-01-01 00:00:16,0,2,3,S03E02
6,düstere Musik,1900-01-01 00:01:41,1900-01-01 00:01:46,1,2,3,S03E02
18,bedrohliche Streichermusik,1900-01-01 00:06:39,1900-01-01 00:06:43,6,2,3,S03E02
46,düstere Musik,1900-01-01 00:20:23,1900-01-01 00:20:27,20,2,3,S03E02
94,Musik verstummt,1900-01-01 00:48:14,1900-01-01 00:48:15,48,2,3,S03E02
...,...,...,...,...,...,...,...
2433,düstere Musik,1900-01-01 00:29:39,1900-01-01 00:29:41,29,9,1,S01E09
2435,Musik verstummt,1900-01-01 00:29:59,1900-01-01 00:30:01,29,9,1,S01E09
2467,melancholische Musik,1900-01-01 00:40:46,1900-01-01 00:40:48,40,9,1,S01E09
2473,melancholische Musik,1900-01-01 00:45:51,1900-01-01 00:45:53,45,9,1,S01E09


In [199]:
from matplotlib.pyplot import legend


fig = px.timeline(data, x_start='interval_start',
                            x_end='interval_end',
                            y='episode',
                            color='season',
                            text='sdh_comment'
                            # ,template="xgridoff"
                            ,template='plotly_dark'
                            ,color_discrete_map={"3":"rgb(179,205,227)","2":"rgb(36,121,108)","1":"rgb(153,153,51)"}
                            ,category_orders={"season": ["1", "2", "3"],"episode": ["10","9","8","7","6","5","4","3","2","1"]}                            
                            ,hover_name="sdh_comment"
                            ,hover_data={'sdh_comment':False,'interval_end':False}
                            ,height=700
                            )

fig.update_xaxes(tickformat="%H:%M:%S",
                showgrid=True, griddash='dot', gridcolor='grey',
                rangeslider=dict(visible=True,
                # ,bgcolor="white"
                )
                ,rangeslider_thickness = 0.1 #fraction of the whole fig heigth
                )
# https://plotly.com/python/reference/layout/xaxis/#layout-xaxis-rangeslider

fig.update_yaxes(autorange="reversed"
                # ,categoryorder='category ascending'
                )

# shapes were added in the background for more readability
for episode in range(0,10):
    fig.add_hrect(y0=episode, y1=episode, line_width=30, fillcolor="black", opacity=0.03,layer="below")

fig.update_traces(textfont_size=8, textangle=0,textposition="none")
fig.update_layout(title="Music of Dark",xaxis_title="Time",yaxis_title="Episode",legend_title="Season",bargap=0,
                font_family="Courier New",
                font_color="white",
                # title_font_family="Times New Roman",
                # title_font_color="white"
                )
fig.show()

In [231]:
# RELAUNCH CELLS WITH DATA IMPORT AND TYPE CHANGES BEFORE PROCEEDING WITH THIS

data = data[(data.sdh_comment.str.startswith('Lied:')) & (~data.sdh_comment.str.contains('Goodbye'))]
# data[data.sdh_comment.str.contains('Goodbye')]
data.episode = data.episode.astype('str')
data.season = data.season.astype('str')
data

,sdh_comment,interval_start,interval_end,interval_minute,episode,season,season00episode00
11,"Lied: ""You Spin Me 'Round"" von Dead or Alive",1900-01-01 00:05:08,1900-01-01 00:05:11,5,2,3,S03E02
93,"Lied: ""Is This Love"" von Survivor",1900-01-01 00:48:03,1900-01-01 00:48:05,48,2,3,S03E02
95,"Lied: ""Broken Sleep"" von Obel",1900-01-01 00:48:45,1900-01-01 00:48:48,48,2,3,S03E02
116,"Lied: ""I Ran"" von A Flock of Seagulls",1900-01-01 00:05:19,1900-01-01 00:05:22,5,7,1,S01E07
313,"Lied: ""Ent One"" von Sol Seppy",1900-01-01 00:40:56,1900-01-01 00:40:58,40,6,1,S01E06
492,"Lied: ""Bad Kingdom"" von May und Robot Koch",1900-01-01 00:49:04,1900-01-01 00:49:06,49,1,3,S03E01
564,"Lied: ""Keep the Streets Empty For Me"" von Fev Ray",1900-01-01 00:38:06,1900-01-01 00:38:09,38,4,1,S01E04
664,"Lied: ""A Quiet Life"" von Teho Teardo & Blixa B...",1900-01-01 00:45:22,1900-01-01 00:45:26,45,10,1,S01E10
742,"Lied: ""Wenn ich dich seh"" von Detlev Lais",1900-01-01 00:04:55,1900-01-01 00:04:58,4,4,3,S03E04
799,"Lied: ""The Labyrinth Song"" von Asaf Avidan",1900-01-01 00:53:53,1900-01-01 00:53:55,53,4,3,S03E04


In [232]:
from matplotlib.pyplot import legend


fig = px.timeline(data, x_start='interval_start',
                            x_end='interval_end',
                            y='episode',
                            color='season',
                            text='sdh_comment'
                            # ,template="xgridoff"
                            ,template='plotly_dark'
                            ,color_discrete_map={"3":"rgb(179,205,227)","2":"rgb(36,121,108)","1":"rgb(153,153,51)"}
                            ,category_orders={"season": ["1", "2", "3"],"episode": ["10","9","8","7","6","5","4","3","2","1"]}                            
                            ,hover_name="sdh_comment"
                            ,hover_data={'sdh_comment':False,'interval_end':False}
                            ,height=700
                            )

fig.update_xaxes(tickformat="%H:%M:%S",
                showgrid=True, griddash='dot', gridcolor='grey',
                rangeslider=dict(visible=True,
                # ,bgcolor="white"
                )
                ,rangeslider_thickness = 0.1 #fraction of the whole fig heigth
                )
# https://plotly.com/python/reference/layout/xaxis/#layout-xaxis-rangeslider

fig.update_yaxes(autorange="reversed"
                # ,categoryorder='category ascending'
                )

# shapes were added in the background for more readability
for episode in range(0,10):
    fig.add_hrect(y0=episode, y1=episode, line_width=30, fillcolor="black", opacity=0.03,layer="below")

fig.update_traces(textfont_size=8, textangle=0,textposition="none")
fig.update_layout(title="Music of Dark",xaxis_title="Time",yaxis_title="Episode",legend_title="Season",bargap=0,
                font_family="Courier New",
                font_color="white",
                # title_font_family="Times New Roman",
                # title_font_color="white"
                )
fig.show()

Now, let's go back to the idea of clustering the comments or something

*Ideas*

- look at what sorts of music is used in the series (ominous, dark, melancholic, and so on)
- cluster comments in some way, I'm assuming there will be categories like sounds, movements, human and animal noize
- 

In [192]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thomasniets/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thomasniets/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import fasttext.util
fasttext.util.download_model('de', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
data = pd.read_csv('sdh_comments.csv')

# changing string type to datetime
data.interval_start = pd.to_datetime(data.interval_start.str[0:8],format="%H:%M:%S")
data.interval_end = pd.to_datetime(data.interval_end.str[0:8],format="%H:%M:%S")

In [237]:
# contains soundtrack titles - no need to include them
data = data[~data.sdh_comment.str.startswith('Lied:')]